In [8]:
import numpy as np
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import json

In [10]:
f = open("Two-group_data.0000", "r")
Input = f.readlines()
f.close()

group_data = np.ones((len(Input),3))
for c, i in enumerate(Input):
    group = Input[c].split(" ")
    group_data[c,0] = float(group[0])
    group_data[c,1] = float(group[1])
    group_data[c,2] = float(group[2])

In [11]:
print(group_data)

[[0.0092 0.0046 1.    ]
 [0.0932 0.1139 0.    ]]


In [16]:
g = open("Two group scattering.0000", "r")
Input2 = g.readlines()
g.close()


scat_list = []
for i in range(len(Input2)):
    scat_list.append(Input2[i].split(" "))
scat_data = np.zeros((len(Input),len(scat_list[0])))
for i in range(len(scat_list)):
    for j in range(len(scat_list[0])):
        scat_data[i,j] = float(scat_list[i][j])

In [17]:
print(scat_data)

[[1.     0.    ]
 [0.0202 2.    ]]


In [18]:
def Create_fission_matrix(group_data):
    dis = len(group_data)
    fission_matrix = np.ones((dis,dis))
    for i in range(dis):
        for j in range(dis):
            fission_matrix[i,j] = group_data[i,2] * group_data[i,1]
    return fission_matrix

In [19]:
Create_fission_matrix(group_data)

array([[0.0046, 0.0046],
       [0.    , 0.    ]])

In [23]:
def Create_absorbtion_matrix(group_data):
    dis = len(group_data)
    absorbtion_matrix = np.zeros((dis,dis))
    for i in range(dis):
        absorbtion_matrix[i][i] = group_data[i,0]
    return absorbtion_matrix

In [24]:
Create_absorbtion_matrix(group_data)

array([[0.0092, 0.    ],
       [0.    , 0.0932]])

In [29]:
def Inscattering_matrix(scat_data):
    Inscattering_matrix = np.zeros ((len(scat_data),len(scat_data[0])))
    for i in range(len(scat_data)):
        for j in range(len(scat_data.T)):
            if i ==j:
                continue
            else:
                Inscattering_matrix[j,i] = scat_data[i,j]
    return Inscattering_matrix

In [30]:
Inscattering_matrix(scat_data)

array([[0.    , 0.0202],
       [0.    , 0.    ]])

In [33]:
def Outscattering_matrix(scat_data):
    Outscattering_matrix = np.zeros((len(scat_data),len(scat_data[0])))
    for i in range(-1,len(scat_data)-1):
        Outscattering_matrix[i,i] = scat_data[i,i+1]
    return Outscattering_matrix

In [34]:
Outscattering_matrix(scat_data)

array([[0.    , 0.    ],
       [0.    , 0.0202]])

In [54]:
k = np.zeros(100)
k[0] = 1
flux = np.zeros((len(scat_data),100))
flux[:,0] = np.ones(len(scat_data))

fission_matrix = Create_fission_matrix(group_data)
absorbtion_matrix = Create_absorbtion_matrix(group_data)
outscattering_matrix = Outscattering_matrix(scat_data)
inscattering_matrix = Inscattering_matrix(scat_data)

mitigation_matrix = absorbtion_matrix + outscattering_matrix - inscattering_matrix

b_matrix = la.inv(mitigation_matrix) @ fission_matrix

eigenvalues, eigenvectors = la.eig(b_matrix)

In [51]:
def power_iteration(M, x):
    # Perform power iteration and return steady state vector xstar
    xc = x.copy()
    A = np.linalg.matrix_power(M,20)
    xc = A @ xc
    return xc

def Power_iteration(A):
    B = A.copy
    for c,i in enumerate(A.T):
        B[:,c] = i/la.norm(i)
    return la.matrix_power(B,100) @ (np.ones(len(A))/len(A))

In [52]:
power_iteration(absorbtion_matrix,[0.5,0.5])

array([9.43466646e-42, 1.22261087e-21])

In [55]:
for i in range(99):
    flux[i+1] = (b_matrix @ flux[i]) / la.norm(b_matrix @ flux[i])
for i in range(99):
    k[i+1] = ((b_matrix @ flux[i+1]).T @ flux[i+1])/(flux[i+1].T @ flux[i+1])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 2)